In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-w_ka8ca4
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-w_ka8ca4
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.0-cp36-none-any.whl size=11716 sha256=66e179efd407e405428252a8cc38d2ef914103caea62a4f2b812bcf470c0a1b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-5442qa3l/wheels/a8/18/22/90afa4bd43247fb9a75b710a4a3fcd94966c022ce9e3c7d0a6
Successfully built preprocess-kgptalkie


In [ ]:
import pandas as pd
import tensorflow as tf
import string
import requests
import numpy as np
import re
from sklearn.model_selection import train_test_split
import preprocess_kgptalkie as ps
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Dropout,LSTM
from tensorflow.keras.layers import Conv1D,MaxPool1D,GlobalMaxPooling1D,MaxPooling1D

In [ ]:
  df = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')

In [ ]:
df.text

'Looking for some education\nMade my way into the night\nAll that bullshit conversation\nBaby, can\'t you read the signs? I won\'t bore you with the details, baby\nI don\'t even wanna waste your time\nLet\'s just say that maybe\nYou could help me ease my mind\nI ain\'t Mr. Right But if you\'re looking for fast love\nIf that\'s love in your eyes\nIt\'s more than enough\nHad some bad love\nSo fast love is all that I\'ve got on my mind Ooh, ooh\nOoh, ooh Looking for some affirmation\nMade my way into the sun\nMy friends got their ladies\nAnd they\'re all having babies\nI just wanna have some fun I won\'t bore you with the details, baby\nI don\'t even wanna waste your time\nLet\'s just say that maybe\nYou could help me ease my mind\nI ain\'t Mr. Right But if you\'re looking for fast love\nIf that\'s love in your eyes\nIt\'s more than enough\nI\'ve had some bad love\nSo fast love is all that I\'ve got on my mind Ooh, ooh\nBaby, baby\nOoh, ooh\nBaby, baby\nOoh, ooh\nBaby, baby\nOoh In the ab

In [ ]:
df = df.text.splitlines()
len(df)


2400

In [ ]:
len(' '.join(df))

91330

In [ ]:

#Tokenization
token = Tokenizer()
token.fit_on_texts(df)

In [ ]:
encoded_text = token.texts_to_sequences(df)
# encoded_text

In [ ]:
# token.word_counts
# token.word_index

In [ ]:
vocab_size = len(token.word_counts) +1

In [ ]:
# Prepare Training data
datalist = []
for d in encoded_text:
  if len(d)>1:
    for i in range(2,len(d)):
      datalist.append(d[:i])
      print(d[:i])

Streaming output truncated to the last 5000 lines.
[197, 203, 438, 16, 1167]
[27, 6]
[27, 6, 218]
[27, 6, 218, 77]
[27, 6, 218, 77, 27]
[27, 6, 218, 77, 27, 6]
[27, 6, 218, 77, 27, 6, 218]
[27, 6, 218, 77, 27, 6, 218, 77]
[27, 6, 218, 77, 27, 6, 218, 77, 21]
[27, 6]
[27, 6, 218]
[27, 6, 218, 77]
[27, 6, 218, 77, 27]
[27, 6, 218, 77, 27, 6]
[27, 6, 218, 77, 27, 6, 218]
[27, 6, 218, 77, 27, 6, 218, 77]
[27, 6, 218, 77, 27, 6, 218, 77, 21]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11]
[27, 6]
[27, 6, 218]
[27, 6, 218, 77]
[27, 6, 218, 77, 27]
[27, 6, 218, 77, 27, 6]
[27, 6, 218, 77, 27, 6, 218]
[27, 6, 218, 77, 27, 6, 218, 77]
[27, 6, 218, 77, 27, 6, 218, 77, 21]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129, 8]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129, 8, 158]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129, 8, 158, 243]
[27, 6, 218, 77, 27, 

In [ ]:
#Padding
max_len = 20
sequence = pad_sequences(datalist, maxlen=max_len,padding='pre')
sequence

array([[  0,   0,   0, ...,   0, 254,  21],
       [  0,   0,   0, ..., 254,  21, 219],
       [  0,   0,   0, ...,   0, 117,   8],
       ...,
       [  0,   0,   0, ...,  17, 198,  17],
       [  0,   0,   0, ..., 198,  17, 198],
       [  0,   0,   0, ...,  17, 198,   6]], dtype=int32)

In [ ]:
sequence.shape


(14231, 20)

In [ ]:
X = sequence[:,:-1]
X

array([[  0,   0,   0, ...,   0,   0, 254],
       [  0,   0,   0, ...,   0, 254,  21],
       [  0,   0,   0, ...,   0,   0, 117],
       ...,
       [  0,   0,   0, ..., 198,  17, 198],
       [  0,   0,   0, ...,  17, 198,  17],
       [  0,   0,   0, ..., 198,  17, 198]], dtype=int32)

In [ ]:
y = sequence[:,-1]
y

array([ 21, 219,   8, ...,  17, 198,   6], dtype=int32)

In [ ]:
y = to_categorical(y,num_classes=vocab_size,)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
seq_len = X.shape[1]
seq_len

19

In [ ]:
#LSTM training
model = Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_len))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            69800     
_________________________________________________________________
lstm (LSTM)                  (None, 19, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 1396)              140996    
Total params: 361,696
Trainable params: 361,696
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X,y,batch_size=32,epochs=100)

Epoch 1/100
445/445 [==============================] - 3s 7ms/step - loss: 5.6917 - accuracy: 0.0424
Epoch 2/100
445/445 [==============================] - 3s 7ms/step - loss: 5.3235 - accuracy: 0.0514
Epoch 3/100
445/445 [==============================] - 3s 7ms/step - loss: 5.1075 - accuracy: 0.0590
Epoch 4/100
445/445 [==============================] - 3s 7ms/step - loss: 4.9159 - accuracy: 0.0715
Epoch 5/100
445/445 [==============================] - 3s 7ms/step - loss: 4.7647 - accuracy: 0.0846
Epoch 6/100
445/445 [==============================] - 3s 7ms/step - loss: 4.6379 - accuracy: 0.0919
Epoch 7/100
445/445 [==============================] - 3s 7ms/step - loss: 4.5183 - accuracy: 0.1056
Epoch 8/100
445/445 [==============================] - 3s 6ms/step - loss: 4.3883 - accuracy: 0.1209
Epoch 9/100
445/445 [==============================] - 3s 6ms/step - loss: 4.2577 - accuracy: 0.1363
Epoch 10/100
445/445 [==============================] - 3s 6ms/step - loss: 4.1157 - accura

In [ ]:
#Poetry Generation
poerty_len = 10

def gen_poerty(seed_text,n_lines):
  for i in range(n_lines):
    text = []
    for _ in range(poerty_len):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded,maxlen=max_len,padding='pre')
      y_pred = np.argmax(model.predict(encoded),axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' '+ predicted_word
      text.append(predicted_word)
    seed_text = text[-1]
    text = ' '.join(text)
    print(text)

In [ ]:
seed_text = 'love'
gen_poerty(seed_text,5)

to you ooh just fun i won't let you close
my head lay down with me words you get so
i spend my whole life hiding my heart away i
can't help myself from looking for you i set fire
burning and it's lighting me first love when it felt
